# Pulp-Smash Internals

### Why?

Get familiar with Pulp-Smash, and get to know how Pulp is tested. 

## Using Jupyter Notebook

You can also follow along using a **Jupyter Notebook**.  Assuming you are bringing your own Pulp machine. 


``` shell
python3 -m venv pulp-smash-internals
source ~/pulp-smash-internals/bin/activate
pip install pulp-smash
pip install jupyter
# Create settings related to the system to be tested
pulp-smash settings create
mkdir code && cd code
git clone https://github.com/kersommoura/Internals.git
cd Internals
jupyter notebook

```

### What is Pulp?

Pulp is a platform for managing repositories of software packages and making it available to a large numbers of consumers. Pulp can locally mirror all or part of a repository, host your own software packages in repositories, and manage many types of content from multiple sources in one place.

https://pulpproject.org/

### Pulp Versions

There are currently 2 versions of Pulp being tested by Pulp-Smash.

* **Pulp2** - As of this writing 2.15.1 is the stable release.

    [Pulp 2 Releases](https://pulp.plan.io/projects/pulp/wiki/Release_Schedule#Past-Releases)


* **Pulp3** - Under development - Release Alpha.

### What is Pulp-Smash?

Pulp-Smash is functional test suite written in Python, to help test Pulp Project. Pulp-Smash is currently used to test Pulp2, and Pulp3. 

Most of the new tests are for Pulp 3.


Smash will decide which tests to run based on several factors, including its configuration file and the state of issues on the Pulp issue tracker.

###  Types of Test

Definitions per **ISTQB**

Unit Testing or Component Testing

> A minimal software item that can be tested in isolation. This activity typically belongs to developers.

Functional Testing

> Basically the testing of the functions of component or system is done. It refers to activities that verify a specific action or function of the code. Functional test tends to answer the questions like “can the user do this” or “does this particular feature work”. This is typically described in a requirements specification or in a functional specification.


### Installing Pulp-Smash 

There are 2 main possible ways to install pulp-smash: as a **user** and a **dev**.

Both options will be covered in this walkthrough.

Resources:

* https://github.com/PulpQE/pulp-smash
* https://pypi.python.org/pypi/pulp-smash
* https://pulp-smash.readthedocs.io/

### Installing Pulp-Smash as a User

Pulp-Smash supports Python 3.4, 3.5 and 3.6 version.

It is suggested the use of a virutalenv to create a sandbox.

``` bash
python3 -m venv ps_env
source ~/.ps_env/bin/activate
pip install pulp-smash
```

After the installation, there is a need to create a configuration file, this configuration file will held all the need information that Pulp-Smash needs to talk with Pulp.
Parameters such as: Pulp version, hostname, if Pulp is available over HTTPS, and so on.

### How to create Pulp-Smash settings

``` bash
 pulp-smash settings create
```
```
pulp-smash settings create
A settings file already exists. Continuing will override it.
Do you want to continue? [y/N]: y
Pulp admin username [admin]: admin
Pulp admin password [admin]: admin
Pulp version: 2.15.1
System hostname: r74-p215
Is Pulp's API available over HTTPS (no for HTTP)? [Y/n]: y
Verify HTTPS? [Y/n]: n
By default, Pulp Smash will communicate with Pulp's API on the port number implied by the scheme. For example, if Pulp's API is available over HTTPS, then Pulp Smash will communicate on port 443.If Pulp's API is avaialable on a non-standard port, like 8000, then Pulp Smash needs to know about that.
Pulp API port number [0]: 0
Is Pulp's message broker Qpid (no for RabbitMQ)? [Y/n]: y
Are you running Pulp Smash on the Pulp system? [y/N]: n
Pulp Smash will be configured to access the Pulp system using SSH. Because of that, some additional information is required.
SSH username to use [root]: root
Ensure ~/.ssh/controlmasters/ exists, and ensure the following is present in your ~/.ssh/config file:

  Host r74-p215
      StrictHostKeyChecking no
      User root
      UserKnownHostsFile /dev/null
      ControlMaster auto
      ControlPersist 10m
      ControlPath ~/.ssh/controlmasters/%C

Creating the settings file at /home/kersom/.config/pulp_smash/settings.json...
Settings file created, run `pulp-smash settings show` to show its contents.
```

### Example of Pulp-Smash settings

``` bash
pulp-smash settings show

```

``` json
{
  "pulp": {
    "auth": [
      "admin",
      "admin"
    ],
    "version": "2.15.1"
  },
  "systems": [
    {
      "hostname": "r74-p215",
      "roles": {
        "amqp broker": {
          "service": "qpidd"
        },
        "api": {
          "scheme": "https",
          "verify": false
        },
        "mongod": {},
        "pulp celerybeat": {},
        "pulp cli": {},
        "pulp resource manager": {},
        "pulp workers": {},
        "shell": {
          "transport": "ssh"
        },
        "squid": {}
      }
    }
  ]
}

```

### Note

The configuration (via `pulp-smash settings create`) is  important. But that comes after installing. And the instructions are the same regardless of how Smash is installed - user or dev.

Settings file is split into a **pulp** and **systems** section because Pulp can be installed across separate hosts. The application-wide settings are placed into the **pulp** section, and the per-host settings are placed into the **systems** section. We don't currently perform multi-host testing of Pulp, but Smash is designed with this use case in mind, and this is one example of that.

Perhaps a more precise definition of **systems** section is to think as **hosts**.


### Good to know

In the simplest case, Pulp Smash's configuration file resides at
    ``~/.config/pulp_smash/settings.json``. However, there are several ways to
    alter this path. Pulp Smash obeys the `XDG Base Directory Specification`_.
    In addition, Pulp Smash responds to the ``PULP_SMASH_CONFIG_FILE``
    environment variable. This variable is a relative path, and it defaults to
    ``settings.json``.


### Running your first test

In [2]:
!python -m unittest pulp_smash.tests.pulp2.platform.api_v2.test_login

s
----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK (skipped=1)


**Warning Explanation**

If the configuration file states that HTTPS connections should not be verified, then InsecureRequestWarnings will be emitted. Smash is smart enough to suppress these warnings. But the unittest test runner overrides Smash and causes all warnings to be emitted anyway.


[Warning](https://github.com/PulpQE/pulp-smash/blob/2c5445972ff8c13cf32632f58b10b787ce2239cd/pulp_smash/tests/__init__.py#L43)


### Running all the tests 

All tests can be run by the following command. This action can take a while.

``` bash
    python -m unittest discover pulp_smash.tests
```


### Installing Pulp-Smash in Development Mode

In development mode changes to code are reflected in the working enviroment.


``` bash
git clone https://github.com/PulpQE/pulp-smash.git
cd pulp-smash
pip install --editable .[dev]
```

### Pulp-Fixtures

Test environment and repeatability.

https://github.com/PulpQE/pulp-fixtures


###  3 Main Components to be explained


Here are the three most important classes that you will encounter as a test
writer:



  **pulp_smash.config.PulpSmashConfig**

 This object stores information about Pulp application and its constituent
 hosts.
 A single Pulp application may have its services spread across several hosts. For example, one host might run Qpid, another might run MongoDB, and so on. Here's how to model a multi-host deployment where Apache runs on one host, and the remaining components run on another .

  **pulp_smash.cli.Client**

 This class provides the ability to execute shell commands on either the
 local host or a remote host. This class provides the ability to execute shell commands on either the local host or a remote host.
 
 **pulp_smash.api.Client**

 A convenience object for working with an API. This class is a wrapper around the `requests.api` module provided by `Requests`.


### Interact with Pulp using Pulp-Smash

Pulp-Smash has 2 main possible ways to interact with Pulp. Using CLI, either REST API.

There is a client for each of these cases.

In [5]:
from pulp_smash.config import PulpSmashConfig

In [6]:
PulpSmashConfig()

PulpSmashConfig(pulp_auth=None, pulp_version='None', systems=[])

### Good to know

1. Instances of PulpSmashConfig() are structured similarly to the config file that is created using **pulp-smash settings create** .
2. pulp_auth and pulp_version correspond to the **pulp** section, and systems corresponds to the **systems** section
3. Some of the differences between the JSON file and this class are worth pointing out. For example, the cfg.pulp_version attribute is a **Version** object, *not* a string.
4. A useful method **get_systems()** for multi-hosts testing.

```python

    def get_systems(self, role):
        """Return a list of hosts fulfilling the given role.
        :param role: The role to filter the available hosts, see
            `pulp_smash.config.ROLES` for more information.
        """
        if role not in ROLES:
            raise ValueError(
                'The given role, {}, is not recognized. Valid roles are: {}'
                .format(role, ROLES)
            )
    return [system for system in self.systems if role in system.roles]

    
```





In [3]:
from pulp_smash.config import get_config

``` python
def get_config():
    """Return a copy of the global ``PulpSmashConfig`` object.
    This method makes use of a cache. If the cache is empty, the configuration
    file is parsed and the cache is populated. Otherwise, a copy of the cached
    configuration object is returned.
    :returns: A copy of the global server configuration object.
    :rtype: pulp_smash.config.PulpSmashConfig
    """
    global _CONFIG  # pylint:disable=global-statement
    if _CONFIG is None:
        _CONFIG = PulpSmashConfig().read()
return deepcopy(_CONFIG)

```

In [9]:
cfg = get_config()

In [10]:
cfg

PulpSmashConfig(pulp_auth=['admin', 'admin'], pulp_version='2.15.1', systems=[PulpSystem(hostname='r74-p215', roles={'amqp broker': {'service': 'qpidd'}, 'api': {'scheme': 'https', 'verify': False}, 'mongod': {}, 'pulp celerybeat': {}, 'pulp cli': {}, 'pulp resource manager': {}, 'pulp workers': {}, 'shell': {'transport': 'ssh'}, 'squid': {}})])

## Client API

Working with an API can require repetitive calls to perform actions like check HTTP status codes.
In addition, Pulp's API has specific quirks surrounding its handling of href paths and HTTP 202 status codes. 
This module provides a customizable client that makes it easier to work with the API in a safe and concise manner.

As mentioned before this class is a wrapped around the `requests.api` module provided by `Requests`. Each of the functions from that moudule are exposed as methods here, and each of the arguments accepted by Requests functions are also accepted by these methods. 

In [11]:
from pulp_smash.api import Client

In [12]:
cfg.get_base_url()

'https://r74-p215'

Worth to be mentioned that Client passes certain arguments by default.

``` python
        """Initialize this object with needed instance attributes."""
        if not pulp_system:
            pulp_system = server_config.get_systems('api')[0]
        self.pulp_system = pulp_system
        self._cfg = server_config
        self.request_kwargs = self._cfg.get_requests_kwargs(pulp_system)
        self.request_kwargs['url'] = self._cfg.get_base_url(pulp_system)
        self.request_kwargs.update(
            {} if request_kwargs is None else request_kwargs
        )

```

In [13]:
client = Client(cfg)

Anything accepted by the `Requests` functions may be placed in ``request_kwargs`` or passed in to a specific call. Example below.

``` python
client.request_kwargs['url'] == 'https://example.com'
client.request_kwargs['verify'] == '~/Documents/my.crt'
```

This possibility has been explored for Pulp3 tests, for instance.

``` python
client.request_kwargs['auth'] = get_auth()

```



In [15]:
response = client.post('/pulp/api/v2/users/', {'login': 'pulp_user4'})
# A new user has to be created every time

/home/kersom/environments/env/lib64/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [17]:
response.json()

{'_href': '/pulp/api/v2/users/pulp_user4/',
 '_id': {'$oid': '5a8af4b6cc5d5e04529ad55a'},
 '_ns': 'users',
 'id': '5a8af4b6cc5d5e04529ad55a',
 'login': 'pulp_user4',
 'name': 'pulp_user4',
 'roles': []}

### Reponse Handlers

Each **Client** object has a callback function, `response_handler`, that is given a chance to munge each server reponse.

Pulp-Smash ships with several response handlers. See:

    pulp_smash.api.code_handler
    pulp_smash.api.echo_handler
    pulp_smash.api.json_handler
    pulp_smash.api.safe_handler
    
There are specific use cases for each of these. All response handlers verify if an exception **HTTPError** was raised, calling `raise_for_status`.

Then when creating the client, the `response_handler` can be defined according to the necessity.

Default : **safe_handler**


In [18]:
from pulp_smash import api

In [19]:
api_client = Client(cfg, api.json_handler)

In [20]:
from pulp_smash.tests.pulp2.rpm.api_v2.utils import gen_repo
from pulp_smash.constants import RPM_UNSIGNED_FEED_URL
from pulp_smash.tests.pulp2.constants import REPOSITORY_PATH

In [21]:
body = gen_repo()  
body['importer_config']['feed'] = RPM_UNSIGNED_FEED_URL
repo = api_client.post(REPOSITORY_PATH, body)

In [22]:
repo = api_client.get(repo['_href'], params = {'details':True})

In [23]:
repo

{'_href': '/pulp/api/v2/repositories/34713c16-827f-4d4b-a26c-04574ecce968/',
 '_id': {'$oid': '5a8af4cbcc5d5e04529ad55d'},
 '_ns': 'repos',
 'content_unit_counts': {},
 'description': None,
 'display_name': '34713c16-827f-4d4b-a26c-04574ecce968',
 'distributors': [],
 'id': '34713c16-827f-4d4b-a26c-04574ecce968',
 'importers': [{'_href': '/pulp/api/v2/repositories/34713c16-827f-4d4b-a26c-04574ecce968/importers/yum_importer/',
   '_id': {'$oid': '5a8af4cbcc5d5e04529ad55e'},
   '_ns': 'repo_importers',
   'config': {'feed': 'https://repos.fedorapeople.org/pulp/pulp/fixtures/rpm-unsigned/'},
   'id': 'yum_importer',
   'importer_type_id': 'yum_importer',
   'last_override_config': {},
   'last_sync': None,
   'last_updated': '2018-02-19T16:01:15Z',
   'repo_id': '34713c16-827f-4d4b-a26c-04574ecce968',
   'scratchpad': None}],
 'last_unit_added': None,
 'last_unit_removed': None,
 'locally_stored_units': 0,
 'notes': {'_repo-type': 'rpm-repo'},
 'scratchpad': {},
 'total_repository_units':

## Client CLI

A convenience object for working with a CLI.




In [26]:
from pulp_smash import config, cli, utils

In [27]:
cli_client = cli.Client(config.get_config())

In [28]:
repo_id = utils.uuid4()

In [29]:
cli_client.run('pulp-admin rpm repo create --repo-id {}'.format(repo_id).split())

CompletedProcess(args=['pulp-admin', 'rpm', 'repo', 'create', '--repo-id', 'b42932e6-f5fc-432e-a966-b761760f1c87'], returncode=0, stdout='\x1b\x1bSuccessfully created repository [b42932e6-f5fc-432e-a966-b761760f1c87]\x1b\n\n', stderr='')

### Response Handlers

A callback function. Each time cli_Client executes a command, the result is handed to this callback, and the callback's return value is handed to the user.

Pulp-Smash ships with several response handlers. See:

    pulp_smash.cli.code_handler
    pulp_smash.cli.echo_handler
   

Default: **code_handler**

### Local or SSH

Pulp Smash compares the host name in its configuration file to the host name of the current system, and *that* is what tells it how to run commands.


In [30]:
cli_client = cli.Client(config.get_config(), cli.echo_handler)

In [31]:
repo_id = utils.uuid4()

In [32]:
cli_client.run('pulp-admin rpm repo create --repo-id {}'.format(repo_id).split())

CompletedProcess(args=['pulp-admin', 'rpm', 'repo', 'create', '--repo-id', '3a70f3b2-f576-4eb2-b29b-583427c93f6a'], returncode=0, stdout='\x1b\x1bSuccessfully created repository [3a70f3b2-f576-4eb2-b29b-583427c93f6a]\x1b\n\n', stderr='')

### Extra documentation

https://pulp-smash.readthedocs.io/en/latest/
    
    

### How to contribute to Pulp-Smash?

https://pulp-smash.readthedocs.io/en/latest/about.html#contributing


### Is there something else that we should test?

File a test case.

https://github.com/PulpQE/pulp-smash/issues

### Thank you.